# AlexNet Multi-Task Model for MedMNIST Submission Using Kaggle Input Data

This notebook demonstrates how to load MedMNIST data directly from Kaggle's input folder, build and train a simplified AlexNet-style model for each MedMNIST task, and generate a submission CSV file.  
Each model is trained for up to **100 epochs** with early stopping (monitoring validation loss).  
The submission file is formatted with columns: `id`, `id_image_in_task`, `task_name`, and `label`.

In [1]:
import tensorflow as tf

# Check for available GPUs and enable memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs found and configured:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Please set your runtime to GPU in Kaggle Notebook settings.")

GPUs found and configured: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
!pip install -q medmnist 

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import f1_score
import medmnist
from medmnist import INFO
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# MedMNIST images are 28x28. We expand them to 3 channels.
TARGET_SIZE = (28, 28)  
BATCH_SIZE = 32

In [4]:
def load_npz_data(npz_path):
    """
    Load train, val, and test arrays from a given .npz file.
    Expected keys: 'train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels'
    """
    data = np.load(npz_path)
    train_images = data['train_images']
    train_labels = data['train_labels']
    val_images = data['val_images']
    val_labels = data['val_labels']
    test_images = data['test_images']
    test_labels = data['test_labels']
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_tf_dataset_from_numpy(images, labels, batch_size=BATCH_SIZE, augment=False):
    """
    Convert numpy arrays to a tf.data.Dataset:
      - For MedMNIST, images are 28x28; expand to (28,28,1) then tile to (28,28,3).
      - Normalize to [0, 1].
      - Optionally apply data augmentation.
    """
    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)  # (N,28,28,1)
    if images.shape[-1] == 1:
        images = np.tile(images, (1, 1, 1, 3))       # (N,28,28,3)
    
    images = images.astype(np.float32) / 255.0
    
    def _process(image, label):
        # Images are already 28x28; no resizing is needed.
        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
        return image, label

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.map(_process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(images))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [5]:
# Define the list of tasks (excluding ChestMNIST)
task_names = [
    "pathmnist",
    "dermamnist",      # Note: The correct Python class in INFO is "DermaMNIST"
    "octmnist",
    "pneumoniamnist",
    "retinamnist",
    "breastmnist",
    "bloodmnist",
    "tissuemnist",
    "organamnist",
    "organcmnist",
    "organsmnist"
]

# Use Kaggle input folder; no downloading occurs.
base_path = Path("/kaggle/input/tensor-reloaded-multi-task-med-mnist/data")
task_to_npz = {task: base_path / f"{task}.npz" for task in task_names}

# Load datasets for each task using the MedMNIST API
train_datasets = {}
val_datasets = {}
test_datasets = {}

for task in task_names:
    (train_imgs, train_lbls), (val_imgs, val_lbls), (test_imgs, test_lbls) = load_npz_data(task_to_npz[task])
    train_datasets[task] = (train_imgs, train_lbls)
    val_datasets[task] = (val_imgs, val_lbls)
    test_datasets[task] = (test_imgs, test_lbls)
    print(f"{task}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test samples")

pathmnist: 89996 train, 10004 val, 7180 test samples
dermamnist: 7007 train, 1003 val, 2005 test samples
octmnist: 97477 train, 10832 val, 1000 test samples
pneumoniamnist: 4708 train, 524 val, 624 test samples
retinamnist: 1080 train, 120 val, 400 test samples
breastmnist: 546 train, 78 val, 156 test samples
bloodmnist: 11959 train, 1712 val, 3421 test samples
tissuemnist: 165466 train, 23640 val, 47280 test samples
organamnist: 34581 train, 6491 val, 17778 test samples
organcmnist: 13000 train, 2392 val, 8268 test samples
organsmnist: 13940 train, 2452 val, 8829 test samples


In [6]:
train_datasets_tf = {}
val_datasets_tf = {}
test_datasets_tf = {}

for task in task_names:
    train_imgs, train_lbls = train_datasets[task]
    val_imgs, val_lbls = val_datasets[task]
    test_imgs, test_lbls = test_datasets[task]
    
    train_datasets_tf[task] = create_tf_dataset_from_numpy(train_imgs, train_lbls, batch_size=BATCH_SIZE, augment=True)
    val_datasets_tf[task]   = create_tf_dataset_from_numpy(val_imgs, val_lbls, batch_size=BATCH_SIZE, augment=False)
    test_datasets_tf[task]  = create_tf_dataset_from_numpy(test_imgs, test_lbls, batch_size=BATCH_SIZE, augment=False)
    
    count = sum(1 for _ in test_datasets_tf[task])
    print(f"{task}: Test batches: {count}")

pathmnist: Test batches: 225
dermamnist: Test batches: 63
octmnist: Test batches: 32
pneumoniamnist: Test batches: 20
retinamnist: Test batches: 13
breastmnist: Test batches: 5
bloodmnist: Test batches: 107
tissuemnist: Test batches: 1478
organamnist: Test batches: 556
organcmnist: Test batches: 259
organsmnist: Test batches: 276


In [7]:
def build_alexnet_model(num_classes, input_shape=(28,28,3)):
    model = Sequential([
        tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), padding='same', 
                                 activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage: (For testing purposes)
# num_classes = len(INFO["pathmnist"]['label'])
# alexnet_model = build_alexnet_model(num_classes, input_shape=(28,28,3))
# alexnet_model.summary()

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

models = {}
histories = {}

for task in task_names:
    num_classes = len(INFO[task]['label'])
    print(f"\nTraining AlexNet model for {task} with {num_classes} classes...")
    model_task = build_alexnet_model(num_classes, input_shape=(28,28,3))
    history = model_task.fit(
        train_datasets_tf[task],
        validation_data=val_datasets_tf[task],
        epochs=100,
        callbacks=[early_stopping],
        verbose=1
    )
    models[task] = model_task
    histories[task] = history


Training AlexNet model for pathmnist with 9 classes...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.4515 - loss: 1.4435 - val_accuracy: 0.6900 - val_loss: 0.8266
Epoch 2/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.7057 - loss: 0.8090 - val_accuracy: 0.7896 - val_loss: 0.5659
Epoch 3/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.7699 - loss: 0.6438 - val_accuracy: 0.8243 - val_loss: 0.4858
Epoch 4/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8077 - loss: 0.5451 - val_accuracy: 0.8453 - val_loss: 0.4189
Epoch 5/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8336 - loss: 0.4811 - val_accuracy: 0.8699 - val_loss: 0.3580
Epoch 6/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8517 - loss: 0.4263 - val_accuracy: 0.8595 - val_loss: 0.3951
Epoch 7/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8597 - loss: 0.3939 - val_accuracy: 0.8718 - val_loss: 0.3524
Epoch 8/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8692 -

In [9]:
submission_rows = []
global_id = 0

for task in task_names:
    model_task = models[task]
    preds_list = []
    for images, _ in test_datasets_tf[task]:
        preds = model_task.predict(images)
        preds = np.argmax(preds, axis=1)
        preds_list.append(preds)
    preds_all = np.concatenate(preds_list)
    for idx, pred in enumerate(preds_all):
        submission_rows.append([global_id, idx, task, int(pred)])
        global_id += 1

submission_df = pd.DataFrame(submission_rows, columns=["id", "id_image_in_task", "task_name", "label"])
print("Total submission rows:", len(submission_df))
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━